In [3]:
from azureml.core import Experiment, Workspace, Model, Environment,Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute

In [2]:
ws=Workspace.get(name='azuremlsdk-ws01', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='azuremlsdk-rg01')

In [6]:
datastore=Datastore.register_azure_blob_container(workspace=ws,datastore_name='diabetesdatastore',container_name='diabetes',account_key='vUU8jFjAcXAmXUID475MCQ+C0Fn15wAxIXPPf76k+R30PVCoEBElx4fUCMevI0AXiiQerijgnF/H+ASt1Nm3bA==',
account_name='storagerrg01',create_if_not_exists=True)

In [8]:
dataset=Dataset.Tabular.from_delimited_files(path=[(datastore,'diabetes.csv')])
dataset=dataset.register(workspace=ws,name='diabetes')
dataset_df=dataset.to_pandas_dataframe()

In [11]:
dataset_df.head()
#dataset_df.isnull().sum()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import joblib

experiment=Experiment(ws, 'modeltrainingdiabetes')
run=experiment.start_logging(snapshot_directory=None)
x=dataset_df.iloc[:,:-1].values
y=dataset_df.iloc[:,-1].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
fitted_model=LR.fit(x_train,y_train)
run.log('Accuracy',fitted_model.score(x_test,y_test))
path='./outputs/model.pkl'
joblib.dump(value=fitted_model,filename=path)
run.complete()

c:\Users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
#register model
model=run.register_model(model_name='diabetesmodel',model_path='outputs/model.pkl')

In [16]:
#creating pipeline
import os
experiment_folder='batch_pipeline'
os.makedirs(experiment_folder,exist_ok=True)


In [29]:
from azureml.core import Environment
from azureml.core.runconfig import CondaDependencies
env=Environment('batch_env')
dep=CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],pip_packages=['azureml-defaults','azureml-core'])
env.python.conda_dependencies=dep
env.register(workspace=ws)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "batch_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
    

In [62]:
csv=[(datastore,'batch_dia.csv')]
dataset_batch=Dataset.Tabular.from_delimited_files(path=csv)
dataset_batch=dataset_batch.register(workspace=ws,name='batch_dataset_test',create_new_version=True)
dataset_batch.to_pandas_dataframe()

,1,106,76,0,0_1,37.5,0.197,26,0_2
0,6,190,92,0,0,35.5,0.278,66,1
1,2,88,58,26,16,28.4,0.766,22,0
2,9,170,74,31,0,44.0,0.403,43,1
3,9,89,62,0,0,22.5,0.142,33,0
4,10,101,76,48,180,32.9,0.171,63,0
5,2,122,70,27,0,36.8,0.340,27,0
6,5,121,72,23,112,26.2,0.245,30,0
7,1,126,60,0,0,30.1,0.349,47,1
8,1,93,70,31,0,30.4,0.315,23,0


In [22]:
#compute
cluster_name='cluster1'
compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
compute_target.wait_for_completion(show_output=True)


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [59]:
%%writefile $experiment_folder/batch_diabetes.py

import os
import numpy as np 
#import Model
from azureml.core.model import Model
import joblib

def init():
    global model
    model_path=Model.get_model_path(model_name='diabetesmodel')
    model=joblib.load(model_path)

def run(mini_batch):
    results=[]
    for f in mini_batch:
        #read comma delimited data into an array
        data=np.genfromtxt(f,delimiter=',')
        prediction=model.predict(data.reshape(1,-1))
        results.append("{}: {}".format(os.path.basename(f),prediction[0]))
    return results
    

Writing batch_pipeline/batch_diabetes.py


In [54]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os


# Load the diabetes data
diabetes = pd.read_csv('C:/Users/satya/Downloads/archive (9)/diabetes.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = Datastore.get(ws,'diabetesdatastore')
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Folder created!
Saving files...
files saved!
Uploading files to datastore...


"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 100 files
Uploading batch-data\1.csv
Uploaded batch-data\1.csv, 1 files out of an estimated total of 100
Uploading batch-data\10.csv
Uploaded batch-data\10.csv, 2 files out of an estimated total of 100
Uploading batch-data\100.csv
Uploaded batch-data\100.csv, 3 files out of an estimated total of 100
Uploading batch-data\11.csv
Uploaded batch-data\11.csv, 4 files out of an estimated total of 100
Uploading batch-data\12.csv
Uploaded batch-data\12.csv, 5 files out of an estimated total of 100
Uploading batch-data\13.csv
Uploaded batch-data\13.csv, 6 files out of an estimated total of 100
Uploading batch-data\14.csv
Uploaded batch-data\14.csv, 7 files out of an estimated total of 100
Uploading batch-data\15.csv
Uploaded batch-data\15.csv, 8 files out of an estimated total of 100
Uploading batch-data\16.csv
Uploaded batch-data\16.csv, 9 files out of an estimated total of 100
Uploading batch-data\17.csv
Uploaded batch-data\17.csv, 10 files out of an estimated total 

In [65]:
#batching pipeline
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData
from azureml.data import OutputFileDatasetConfig

output_dir=OutputFileDatasetConfig(name='inferences')
ds=Datastore.get(ws,'diabetesdatastore')
#output_dir=PipelineData(name='inferences',datastore=ds,output_path_on_compute='diabetes/results')

parrallel_run_config=ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script='batch_diabetes.py',
    mini_batch_size="5",
    error_threshold=10,
    output_action='append_row',
    environment=env,
    compute_target=compute_target,
    node_count=1
)

parrallel_run_step=ParallelRunStep(
    name='batch_inference',
    parallel_run_config=parrallel_run_config,
    inputs=[batch_data_set.as_named_input('raw_data')],
    output= output_dir,
    allow_reuse=True
)


In [66]:
from azureml.pipeline.core import Pipeline
pipeline=Pipeline(workspace=ws,steps=[parrallel_run_step])
pipeline_run=pipeline.submit(experiment_name='batch_pipeline')
pipeline_run.wait_for_completion(show_output=True)

Created step batch_inference [935f2595][9358729a-313e-454f-987b-129e6f010275], (This step will run and generate new outputs)
Submitted PipelineRun 69b19816-7b0c-423f-acb5-a35145dcf779
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/69b19816-7b0c-423f-acb5-a35145dcf779?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRunId: 69b19816-7b0c-423f-acb5-a35145dcf779
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/69b19816-7b0c-423f-acb5-a35145dcf779?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRun Status: Running


StepRunId: a618362e-3763-4709-9406-66bdcf7bd0b3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a618362e-3763-4709-9406-66bdcf7bd0b3?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegro

'Finished'